In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader,TensorDataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm

#Importing Metric modules
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score




In [38]:
# Extracted Features path
healthy_features_path = 'featuresvalues_new2D_automated.csv'
tumor_features_path = 'tumorfeaturesvalues_new2D_automated_70.csv'

In [39]:
# Reading the healthy pancreas features
data = pd.read_csv(healthy_features_path)
X_healthy = data.iloc[:, :-1]
y_healthy = data.iloc[:, -1]

In [40]:
# Reading the tumor pancreas features
data_tumor = pd.read_csv(tumor_features_path)
X_tumor = data_tumor.iloc[:, :-1]
y_tumor = data_tumor.iloc[:, -1]

In [41]:
# Concat Features
z=y_tumor.values.reshape(256,1)
h = y_healthy.values.reshape(89,1)
modified_array = np.vstack((z, h))
y = modified_array.reshape(1,345)

In [42]:
z1=X_tumor.values.reshape(256,102)
h1 = X_healthy.values.reshape(89,102)
modified_array = np.vstack((z1, h1))
X = modified_array.reshape(102,345)

In [43]:
X =X.reshape(345,102)
y = y.reshape(345,1)

In [44]:
modified_data = np.hstack((X,y))

In [45]:
df = pd.DataFrame(modified_data)
# df_shuffled

In [48]:
class FNN_model(nn.Module):
    def __init__(self):
        super(FNN_model, self).__init__()
        self.fc1 = nn.Linear(102, 64)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(32, 8)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
#         x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.sigmoid(self.fc4(x))
        return x



In [49]:
def load_pretrained(model, pretrained_path):
    state_dict = torch.load(pretrained_path)
    model.load_state_dict(state_dict)
    print("Pretrained model loaded successfully!")

In [50]:
def train(model,criterion,optimizer,num_epochs,train_loader,val_loader,scheduler,count):
    model.train()
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        epoch_loss = 0  # Initialize the loss for one epoch to zero
        correct_predictions = 0  # Initialize the number of correct predictions

        num_batches = 0 # Helps to report avg epoch loss across all batches
        total_correct = 0
        
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

        for batch_idx, (features,labels) in progress_bar:
            optimizer.zero_grad()

            predictions = model(features.float())
            batch_loss = criterion(predictions,labels.unsqueeze(1).float())
            
            batch_loss.backward()
            optimizer.step()
            
            
            epoch_loss += batch_loss.item()
            num_batches += 1
   
            

            # Update the progress bar description with the average loss
            progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {batch_loss.item():.4f}")
            
            # Validation process
        print("Satrting the Validation process...")
        model.eval() # model set to evaluation mode
        batch_loss_validation = 0
        progress_bar_validation = tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {epoch+1}")

        with torch.no_grad():
            for batch_idx, (features,labels) in progress_bar_validation:
                
                outputs = model(features.float())
                loss = criterion(outputs, labels.unsqueeze(1).float())

                batch_loss_validation += loss.item()
              
                progress_bar_validation.set_description(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {loss.item():.4f}")

        scheduler.step(batch_loss_validation)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}, Validation Loss: {batch_loss_validation/len(val_loader):.4f}")
        resultant_loss = batch_loss_validation/len(val_loader)
        if resultant_loss < best_val_loss:
            best_val_loss = resultant_loss
            # counter = 0
            torch.save(model.state_dict(), f'FNN_Radio_checkpoint_{count}.pth')

    

In [51]:
# Initialize k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds)

In [52]:
iterations = 1

In [53]:
accuracy_list = []
precision_list =[]
recall_list = []
f1_score_list = []
roc_auc_list = []
for it in range(1,iterations+1):
    accuracy_fold = []
    precision_fold =[]
    recall_fold = []
    f1_fold = []
    roc_auc_fold = []
    count = 0
    # Shuffling data every iteration
    df_shuffled = df.sample(frac=1)
    X1= df_shuffled.iloc[:, :-1].values
    y1 = df_shuffled.iloc[:,-1].values


    for train_index,val_index in kf.split(X1):
        count+=1
        print(f"Starting the {count} fold")
        
        model = FNN_model()
        criterion = nn.BCELoss()  # Binary Cross Entropy Loss
        # optimizer = optim.SGD(model.parameters(), lr=0.01)
        optimizer = optim.Adam(model.parameters(),lr=0.001)
        num_epochs = 40
        # Define scheduler for learning rate adjustment
        scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, verbose=True)


        # Split data into training and validation sets
        X_train, X_val = X1[train_index], X1[val_index]
        y_train, y_val = y1[train_index], y1[val_index]

        scaler = MinMaxScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
        
    #     print(type(y_train))

        # Create DataLoader for training and validation sets
        train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
        val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
        train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)

        
        train(model,criterion,optimizer,num_epochs,train_loader,val_loader,scheduler,count)
        
        # Instantiate the model for validation
        model_new = FNN_model()

        # Load pre-trained weights
        pretrained_path = f'FNN_Radio_checkpoint_{count}.pth'
        load_pretrained(model_new, pretrained_path)
        
        model_new.eval()

        # Initialize variables for calculating validation accuracy
        val_correct = 0

        # Disable gradient calculation during validation
        with torch.no_grad():
            # for inputs, labels in val_loader:
                # Forward pass
                outputs = model_new(torch.tensor(X_val).float())
                predicted = torch.round(outputs).squeeze(1).detach().numpy()
                accuracy_fold.append(accuracy_score(y_val, predicted))
                precision_fold.append(precision_score(y_val, predicted))
                recall_fold.append(recall_score(y_val, predicted))
                f1_fold.append(f1_score(y_val, predicted))
                roc_auc_fold.append(roc_auc_score(y_val, predicted))

    # Store mean performance metrics for this iteration
    accuracy_list.append(np.mean(accuracy_fold))
    precision_list.append(np.mean(precision_fold))
    recall_list.append(np.mean(recall_fold))
    f1_score_list.append(np.mean(f1_fold))
    roc_auc_list.append(np.mean(roc_auc_fold))
            
            # # Calculate accuracy for the current batch
            # correct = (predicted == labels.numpy()).sum()
    
            # val_correct += correct
    
    

    # Calculate validation accuracy
    # val_accuracy = val_correct / len(val_dataset)

    


Starting the 1 fold


Epoch 1/40, Loss: 0.6495: 100%|██████████| 138/138 [00:00<00:00, 310.86it/s]


Satrting the Validation process...


Epoch 1/40, Validation Loss: 0.8778: 100%|██████████| 3/3 [00:00<00:00, 501.85it/s]


Epoch 1/40, Loss: 0.5908, Validation Loss: 0.6638


...

Epoch 39/40, Loss: 0.1679, Validation Loss: 0.2383


Epoch 40/40, Loss: 0.1271: 100%|██████████| 138/138 [00:00<00:00, 313.46it/s]


Satrting the Validation process...


Epoch 40/40, Validation Loss: 0.1294: 100%|██████████| 3/3 [00:00<00:00, 541.71it/s]


Epoch 40/40, Loss: 0.2036, Validation Loss: 0.2228
Pretrained model loaded successfully!


In [54]:

print("Printing values for 10 iterations FNN")
# Calculate the mean and std accuracy
mean_accuracy = sum(accuracy_list) / len(accuracy_list)
std_accuracy = np.std(accuracy_list)

# Report the mean accuracy up to 4 decimal points
print(f"Mean Accuracy: {mean_accuracy : .4f}")
print(f"Std Accuracy: {std_accuracy : .4f}")

# Calculate the mean and std precision
mean_precision = sum(precision_list) / len(precision_list)
std_precision = np.std(precision_list)
# Report the mean precision up to 4 decimal points

print(f"Mean Precision: {mean_precision : .4f}")
print(f"Std Precision: {std_precision : .4f}")

# Calculate the mean and std recall
mean_recall = sum(recall_list) / len(recall_list)
std_recall = np.std(recall_list)
# Report the mean recall up to 4 decimal points

print(f"Mean Recall: {mean_recall : .4f}")
print(f"Std Recall: {std_recall : .4f}")

# Calculate the mean and std roc_auc
mean_roc_auc = sum(roc_auc_list) / len(roc_auc_list)
std_roc_auc = np.std(roc_auc_list)
# Report the mean roc_auc up to 4 decimal points

print(f"Mean ROC-AUC: {mean_roc_auc : .4f}")
print(f"Std ROC-AUC: {std_roc_auc : .4f}")

# Calculate the mean and std f1_score
mean_f1_score = sum(f1_score_list) / len(f1_score_list)
std_f1_score = np.std(f1_score_list)
# Report the mean f1-score up to 4 decimal points

print(f"Mean F1-Score: {mean_f1_score : .4f}")
print(f"Std F1-Score: {std_f1_score : .4f}")



Printing values for 10 iterations FNN
Mean Accuracy:  0.8754
Std Accuracy:  0.0000
Mean Precision:  0.8966
Std Precision:  0.0000
Mean Recall:  0.9527
Std Recall:  0.0000
Mean ROC-AUC:  0.8058
Std ROC-AUC:  0.0000
Mean F1-Score:  0.9202
Std F1-Score:  0.0000
